In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

ticker = 'AAPL' #'AAPL'
stock = yf.Ticker(ticker)
balance_sheets = stock.balance_sheet
balance_sheets.columns = pd.to_datetime(balance_sheets.columns)
balance_sheet = balance_sheets.loc[:, '2021-09-30':'2024-09-30'] # we have 4y data
# print(balance_sheet)

In [2]:
balance_sheet = balance_sheet.T
fields_of_interest = [
    'Cash And Cash Equivalents',
    'Accounts Receivable',
    'Inventory',
    'Net PPE',
    'Accounts Payable',
    'Long Term Debt',
    'Stockholders Equity',
    'Retained Earnings'
]

# abstract data
df = balance_sheet[fields_of_interest]

In [3]:
df

,Cash And Cash Equivalents,Accounts Receivable,Inventory,Net PPE,Accounts Payable,Long Term Debt,Stockholders Equity,Retained Earnings
2024-09-30,29943000000.0,33410000000.0,7286000000.0,45680000000.0,68960000000.0,85750000000.0,56950000000.0,-19154000000.0
2023-09-30,29965000000.0,29508000000.0,6331000000.0,43715000000.0,62611000000.0,95281000000.0,62146000000.0,-214000000.0
2022-09-30,23646000000.0,28184000000.0,4946000000.0,52534000000.0,64115000000.0,98959000000.0,50672000000.0,-3068000000.0
2021-09-30,34940000000.0,26278000000.0,6580000000.0,49527000000.0,54763000000.0,109106000000.0,63090000000.0,5562000000.0


In [4]:
y_data = df[['Cash And Cash Equivalents', 'Accounts Receivable', 'Inventory', 
             'Net PPE', 'Accounts Payable', 'Long Term Debt', 'Stockholders Equity', 'Retained Earnings']].values

# 计算 A(t) - L(t) - E(t)
A_L_E = df['Cash And Cash Equivalents'] + df['Accounts Receivable'] + df['Inventory'] + df['Net PPE']
L_E = df['Accounts Payable'] + df['Long Term Debt']
x_data = A_L_E - L_E - df['Stockholders Equity'].values

def create_dataset(x, y, look_back=0):
    X, Y = [], []
    for i in range(len(x) - look_back):
        X.append(x[i:i+look_back])
        Y.append(y[i+look_back])
    return np.array(X), np.array(Y)

# 创建输入和输出数据集
X, Y = create_dataset(x_data, y_data)

# 划分训练集和测试集（例如：80%用于训练，20%用于测试）
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]

print(f"Training data size: {len(X_train)}, Test data size: {len(X_test)}")

Training data size: 3, Test data size: 1


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
model.add(Dense(32, activation='relu'))
model.add(Dense(8))  # 输出层，预测资产负债表中的8个指标

model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
history = model.fit(X_train, Y_train, epochs=50, batch_size=16, validation_data=(X_test, Y_test))

import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

In [ ]:
test_loss = model.evaluate(X_test, Y_test)
print(f"Test Loss: {test_loss}")
y_pred = model.predict(X_test)
print(f"Predicted: {y_pred[:5]}")
print(f"Actual: {Y_test[:5]}")

In [6]:
import yfinance as yf

# 获取 AAPL 的财务数据
ticker = yf.Ticker("AAPL")

# 获取季度收入报表（Income Statement）
income_statement = ticker.quarterly_income_stmt

# 获取季度资产负债表（Balance Sheet）
balance_sheet = ticker.quarterly_balance_sheet

# 获取季度现金流量表（Cash Flow Statement）
cash_flow = ticker.quarterly_cashflow

print("Income Statement Fields:")
print(income_statement.index)  # 或使用 income_statement.columns 以查看列名

print("\nBalance Sheet Fields:")
print(balance_sheet.index)  # 或使用 balance_sheet.columns

print("\nCash Flow Fields:")
print(cash_flow.index)  # 或使用 cash_flow.columns

Income Statement Fields:
Index(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense', 'Interest Income',
       'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Total Operating Income As Reported',
       'Diluted Average Shares', 'Basic Average Shares', 'Diluted EPS',
       'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Net Income',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Income Expense', 'Other Non Operating Income Expenses',
       'Net Non Operating Interest Income Expense',
       'Interest Expense Non Operating', 'Interest Income Non Operating